# Bash

Когда речь заходит об обработке больших данных, на первый план выходят задачи по грамотной манипуляции данными, конвертированию и выгрузке, настройке специализированных инструментов и так далее. Чтобы уметь решать такие задачи с кайфом, нужно обрести уверенные навыки работы в терминале и командных оболочках.

Bash - наиболее популярная командная оболочка в UNIX-like система. Когда вы пользуетесь терминалом на своем компьютере, то вы почти наверное используете именно Bash.

В баш встроено огромное количество функциональности, которая не ограничивается лишь запуском программ.

In [ ]:
## Скачаем данные для семинара. Они потребуются позже

! wget https://people.sc.fsu.edu/~jburkardt/data/csv/grades.csv
! wget https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/data/sem2/covid.json

--2021-05-20 08:18:35--  https://people.sc.fsu.edu/~jburkardt/data/csv/grades.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.16.102
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.16.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1545 (1.5K) [text/csv]
Saving to: ‘grades.csv.1’

grades.csv.1        100%[===================>]   1.51K  --.-KB/s    in 0s      

2021-05-20 08:18:35 (259 MB/s) - ‘grades.csv.1’ saved [1545/1545]

--2021-05-20 08:18:35--  https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/data/sem2/covid.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 517194 (505K) [text/plain]
Saving to: ‘covid.json.1’

covid.json.1        100%[===================>] 505.07K  --.-KB/s    in

# Потоки данных

Сильная сторона языка Bash - это возможности по работе с перенаправлениями потоками данных между подпрограммами.

Для Bash каждую программу можно представить в следующем виде - это черный ящик, которому

На вход подается какой-то источник данных, а также аргументы запуска

На выход выдает два источника данных - это вывод результата и вывод ошибок.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-1.png">

Чаще всего к такому "ящику" подключаются три стандартный источника данных - это stdin, stdout и stderr.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-2.png">

Все эти три источника, как и все в UNIX, являются виртуальными файлами и выполняют следующие функции

* stdin представляет весь поток данных, который пользователь вводит с клавиатуры
* stdout представляет весь поток данных, который программа печатает на экран
* stderr представляет весь поток данных об ошибках в работе программы, который программа также печатает на экран

Все эти источники находятся по следующим путям: `/dev/stdout`, `/dev/stdin`, `/dev/stderr`. Можно заглянуть в директорию `/dev/` и посмотреть, сколько еще виртуальных источников данных есть в компьютере.

In [ ]:
! ls /dev

core  full  mqueue  ptmx  random  stderr  stdout  urandom
fd    fuse  null    pts   shm	  stdin   tty	  zero


Волшебство UNIX заключается в том, что мы можем работать с этими источниками как с самыми обычными файлами. Например, давайте попробуем что-то написать в файл /dev/stdout

In [ ]:
%%writefile hello-stdout.py
with open('/dev/stdout', 'w') as f:
    f.write("HELLO, STDOUT!")  # Пишем в специальный файл, вместо print

Writing hello-stdout.py


In [ ]:
! ls

covid.json    grades.csv    hello-stdout.py
covid.json.1  grades.csv.1  sample_data


In [ ]:
! python3 hello-stdout.py

HELLO, STDOUT!

Вместо стандартных потоков ввода\вывода можно подставлять произвольные источники данных.

**Оператор >** позволяет перенаправлять стандартный вывод в любой другой файл. Попробуем, например, написать что-то в новый файл используя команду echo.

In [ ]:
! echo "jdhujhdjkfhsdkf"

jdhujhdjkfhsdkf


In [ ]:
! echo "message from echo"

message from echo


In [ ]:
! echo "message from echo" > file.txt

In [ ]:
! cat file.txt

message from echo


In [ ]:
! cat file.txt file.txt # Читает файл два раза

message from echo
message from echo


In [ ]:
! cat file.txt file.txt > doubled-file.txt

In [ ]:
! cat doubled-file.txt

message from echo
message from echo


Если файл уже существует, то этот оператор полностью перезатрет его содержимое

In [ ]:
! echo "new message" > doubled-file.txt

In [ ]:
! cat doubled-file.txt

new message


**Оператор >>** позволяет не перезаписывать целиком файл, а лишь добавить в конец новые данные

In [ ]:
! cat file.txt file.txt > doubled-file.txt

In [ ]:
! cat doubled-file.txt

message from echo
message from echo


In [ ]:
! echo "new message" >> doubled-file

In [ ]:
! cat doubled-file

new message


Помимо вывода программы, можно поменять и ее ввод.

**Оператор <** позволяет поменить стандартный ввод программы на другой файл, делая вид для программы, будто бы пользователь ввел эти данные с клавиатуры.

Напишем простенькую программу, которая читает пользовательский ввод с клавиатуры и потестируем этот оператор.

In [ ]:
%%writefile repeat.py

data = input()
for i in range(3):
    print(data)

Writing repeat.py


In [ ]:
! cat file.txt

message from echo


In [ ]:
! python3 repeat.py < file.txt

message from echo
message from echo
message from echo


In [ ]:
! python3 -c "import sys; print(list(sys.stdin))" < doubled-file.txt

['message from echo\n', 'message from echo\n', 'new message\n']


Если хочется направить на stdin несколько строк, но при этом не из файла, а прямо из скрипта, то можно воспользваться **оператором <<**. Для него нужно отдельно указать маркер начала и конца данных.

In [ ]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<END
big
multiline
message
from
script
END

['big\n', 'multiline\n', 'message\n', 'from\n', 'script\n']


Если хочется направить ровно одну строку, то можно воспользоваться **оператором <<<**. Он подаст на stdin ту строку, которую мы передадим в качестве аргумента.

In [ ]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<< "one line message from script"

['one line \\n message from script\n']


Комбинирование этих возможностей может позволить решить некоторые задачи. Например генерировать какой-то файл на лету.

In [ ]:
%%bash

cat > message.txt <<MSG
This file was created directly from script
Using amazing features of Bash
MSG 

bash: line 5: warning: here-document at line 2 delimited by end-of-file (wanted `MSG')


In [ ]:
! cat message.txt

This file was created directly from script
Using amazing features of Bash
MSG 


Или например решать проблему с интерактивными программами, которые хотят какого-то действия от пользователя.

In [ ]:
%%writefile interactive.py

answer = input("Are you sure you want to do X? (y/n)")
if answer == 'y':
    print("DOING X")
else:
    print("Canceling")

Writing interactive.py


In [ ]:
! python3 interactive.py <<< "y"

Are you sure you want to do X? (y/n)DOING X


In [ ]:
! python3 interactive.py <<< "n"

Are you sure you want to do X? (y/n)Canceling


Возможности перенаправлений не ограничиваются только статическими текстовыми файлами. 

**Оператор <()** позворяет представить вывод программы как специальный файл (чаще всего он выглядит как `/dev/fd/63`) . Основная особенность этого оператора от обычного **оператора >** в том, что не используется дополнительное место на диске. Читая из этого специального файла мы напрямую получаем доступ к выводу программы без необходимости сохранять его на диске в явном виде.

Сам оператор при применении запускает переданную программу, создает специальный файл и возращает путь до этого файла.

Этим можно воспользоваться, чтобы компоновать работу нескольких программ.

In [ ]:
! echo <(echo PIPA)  # Напечатали путь до файла, в коротый подключен вывод программы echo PIPA

/dev/fd/63


In [ ]:
! cat <(echo PIPA)

PIPA


In [ ]:
%%writefile read-file.py

import sys

print("Path to file = {}".format(sys.argv[1]))

with open(sys.argv[1], 'r') as f:
    content = f.read()
    print(content)

Writing read-file.py


In [ ]:
! cat file.txt

message from echo


In [ ]:
! python3 read-file.py file.txt

Path to file = file.txt
message from echo



In [ ]:
! python3 read-file.py <(echo PIPA)

Path to file = /dev/fd/63
PIPA



Комбинируя возможности операторов **<** и **<()** можно перенаправлять вывод одной команды в другой.

Наглядный пример - команда `yes`. Она решает уже рассмотренную проблему работы с интерактивными программами, без конца печатая символ `y` на стандартный вывод. 

(Интересный факт - люди любят соревноваться в "производительности" программы `yes`. Согласно [треду на Реддите](https://www.reddit.com/r/unix/comments/6gxduc/how_is_gnu_yes_so_fast/), рекорд - вывод `y` со скоростью 123 Гигабита в секунду. Зачем нужная такая производительность в команде `yes`? Ну чтобы было смешно.)

In [ ]:
! python3 interactive.py < <(yes)

Are you sure you want to do X? (y/n)DOING X


Другой полезный пример - сравнение двух каталогов. Сущестует программа `diff`, которая получает два файла и выводит построчные различия в этий файлах. Можно скомбинировать ее с командой `ls`, чтобы научиться сравнивать директории.

In [ ]:
! mkdir -p folder1
! touch folder1/file1.txt folder1/file2.txt folder1/file3.txt

! mkdir -p folder2
! touch folder2/file2.txt folder2/file3.txt folder2/file4.txt

In [ ]:
! ls folder1

file1.txt  file2.txt  file3.txt


In [ ]:
! ls folder2

file2.txt  file3.txt  file4.txt


In [ ]:
! diff <(ls folder1) <(ls folder2)

1d0
< file1.txt
3a3
> file4.txt


Идея соединять процессы через stdin\stdout очень популярна и для этого есть более удобный интерфейс - pipes или **оператор |**

Все команды, соединенные через | запускаются одновременно и общаются друг с другом через stdin\stdout

In [ ]:
! echo hello | cat

hello


In [ ]:
! echo hi | wc -l  # wc считает сколько строк в входных данных

1


In [ ]:
! cat doubled-file.txt

message from echo
message from echo
new message


In [ ]:
! cat doubled-file.txt | wc -l

3


In [ ]:
! ls -l | wc -l

17


In [ ]:
! ls -l | cat | wc -l | python3 -c "print(int(input()) * 2)"

34


In [ ]:
! yes | python3 interactive.py

Are you sure you want to do X? (y/n)DOING X


In [ ]:
! yes | head  # Смотрим первые 10 строк файла

y
y
y
y
y
y
y
y
y
y


Существует также родственный оператор **$()**. Он также запускает переданную программу, однако перенаправляет вывод не в файл, а прямо в bash. То есть вывод программы можно использовать как строку внутри скрипта.

In [ ]:
! pwd

/content


In [ ]:
! echo I am here - $(pwd)

I am here - /content


In [ ]:
%%writefile file-to-read.txt
/etc/hosts

Writing file-to-read.txt


In [ ]:
! cat $(cat file-to-read.txt) # cat /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


# Полезные программы

Bash - всего лишь оболочка и не умеет самостоятельно решать какие-то задачи. Основную работу выполняют установленные программы, которые можно вызывать из bash. Их можно использовать гораздо эффективнее, используя их вместе с возможностями оболочки.

### Head
head читает определенное количество данных с начала файла. Это полезно, например, когда хочется посмотреть на часть данных, которая лежит на диске.

In [ ]:
! head /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


In [ ]:
! head -n 2 /etc/hosts  # Читаем только первые 2 строки

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback


In [ ]:
! head -c 10 /etc/hosts  # Читаем только первые 10 байт

127.0.0.1	

In [ ]:
! cat /etc/hosts | head -n 2  # Как и почти все программы, которые мы рассмотрим, умеет работать с вводом

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback


In [ ]:
! head -n -2 /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes


### Tail
tail делает то же самое, что и head, но с конца файла

In [ ]:
! tail /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


In [ ]:
! tail -n 2 /etc/hosts

ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


In [ ]:
! tail -n +2 /etc/hosts  # все строки после второй строки (включая вторую строку)

::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


In [ ]:
! cat /etc/hosts | tail -n 2

ff02::2	ip6-allrouters
172.28.0.2	0b0103324ace


### Sort
sort сортирует входные данные. По умолчанию используется лексикографический порядок, но это поведение можно поменять с использованием специальных опций.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=sort&category=1

In [ ]:
%%writefile numbers.txt
3
5
1
2
6
5
9
4
5
6
7
3
2
10
5
6

Writing numbers.txt


In [ ]:
! sort numbers.txt  # Сортируем как строки

1
10
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [ ]:
! cat numbers.txt | sort

1
10
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [ ]:
! cat numbers.txt | sort -n # Сортируем как числа

1
2
2
3
3
4
5
5
5
5
6
6
6
7
9
10


Если в данных есть сразу несколько "колонок" (например в каждой строке есть значения, разделенные пробельным символом), то можно отдельно указать, по какому полю необходимо сортировать

In [ ]:
%%writefile number-table.txt
1 10
2 9
3 8
4 7
5 6
6 5
7 4
8 3
9 2
10 1

Writing number-table.txt


In [ ]:
! cat number-table.txt | sort -k1,1 -n

1 10
2 9
3 8
4 7
5 6
6 5
7 4
8 3
9 2
10 1


In [ ]:
! cat number-table.txt | sort -k2,2 -n

10 1
9 2
8 3
7 4
6 5
5 6
4 7
3 8
2 9
1 10


In [ ]:
! cat numbers.txt | sort -n -r  # Сортируем в обратном порядке

10
9
7
6
6
6
5
5
5
5
4
3
3
2
2
1


### Shuf
shuf напротив, случайным образом перемешивает входящие данные

In [ ]:
! cat numbers.txt | shuf

6
10
3
5
1
2
6
5
2
3
5
6
9
5
7
4


In [ ]:
! cat numbers.txt | shuf

5
6
3
10
4
2
1
3
6
9
5
6
2
5
7
5


### Uniq
uniq оставляет только уникальные значения. Однако он корректно работает только с отсортированными данными. Для этого мы можем предварительно использовать sort.

Помимо операции схлопывания одинаковых значений, uniq также умеет считать простые статистики для схлопнувшихся групп. Этот функционал чем-то напоминает group by. Так, ключ -c считает количество элементов в каждой группе.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=uniq&category=1&russian=0

In [ ]:
! cat numbers.txt | sort

1
10
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [ ]:
! cat numbers.txt | uniq  # Не совсем тот результат, что мы ожидаем

3
5
1
2
6
5
9
4
5
6
7
3
2
10
5
6


In [ ]:
! cat numbers.txt | sort | uniq  # А вот так уже работает

1
10
2
3
4
5
6
7
9


По ходу "схлопывания" uniq умеет еще и подсчитывать количество схлопнутых элементов. Таким образом можно считать количество каждого элемента.

In [ ]:
! cat numbers.txt | sort | uniq -c

      1 1
      1 10
      2 2
      2 3
      1 4
      4 5
      3 6
      1 7
      1 9


In [ ]:
%%writefile words.txt
Lorem
ipsum
dolor
sit
met
consectetur
incididunt
elit
seddo
ipsum
tempor
incididunt
ut
laboret
dolor
ipsum
aliqua
ipsum

Writing words.txt


In [ ]:
! cat words.txt | sort | uniq -c 

      1 aliqua
      1 consectetur
      2 dolor
      1 elit
      2 incididunt
      4 ipsum
      1 laboret
      1 Lorem
      1 met
      1 seddo
      1 sit
      1 tempor
      1 ut


### Wc
wc (word count) считает количество элементов во входных данных. По умолчанию считает три характеристики - количество строк, количество слов, количество байт. Различные опции позволяют считать какую-то одну из характеристик. Например -l считает количество строк в данных.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wc&category=1&russian=0

In [ ]:
! cat numbers.txt | wc

     15      16      32


In [ ]:
! cat number-table.txt | wc

      9      20      41


In [ ]:
! cat numbers.txt | wc -l # Количество элементов в файле

15


In [ ]:
! cat numbers.txt | sort | uniq | wc -l  # Количество уникальных элементов в файле

9


### Cut

cut парсит строки, которые состоят из значений с разделителем. С помощью утилиты можно обрабатывать различные регулярные форматы данных, базирующиеся на разделителях. Например csv или tsv.

https://www.opennet.ru/man.shtml?topic=cut&category=1&russian=0


In [ ]:
! cat grades.csv

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"

In [ ]:
! cat grades.csv | cut -d',' -f4,5,6,7  # разделяем данные по запятой и берем только 6 и 7 столбец

        "Test1", "Test2", "Test3", "Test4"
 40.0,    90.0,   100.0,    83.0
 41.0,    97.0,    96.0,    97.0
 41.0,    80.0,    60.0,    40.0
 42.0,    23.0,    36.0,    45.0
 43.0,    78.0,    88.0,    77.0
 44.0,    90.0,    80.0,    90.0
 45.0,    11.0,    -1.0,     4.0
 46.0,    20.0,    30.0,    40.0
 49.0      1.0,    90.0,   100.0,    83.0
 48.0,     1.0,    97.0,    96.0
 44.0,     1.0,    80.0,    60.0
 47.0,     1.0,    23.0,    36.0
 45.0,     1.0,    78.0,    88.0
 50.0,     1.0,    90.0,    80.0
 40.0,     1.0,    11.0,    -1.0
 30.0,     1.0,    20.0,    30.0


In [ ]:
# удаляем заголовок и смотрим только на grade
! cat grades.csv | tail -n +2 | cut -d',' -f9 | head

   "D-"
   "D+"
   "C"
   "B-"
   "A-"
   "C-"
   "F"
   "B+"

   "A+"


In [ ]:
# Считаем уникальные оценки в графе test1
! cat grades.csv | tail -n +2 | cut -d',' -f4 | sort | uniq | wc -l

12


In [ ]:
! uname -a  # информация о системе

Linux 0b0103324ace 5.4.109+ #1 SMP Tue Apr 20 19:55:43 PDT 2021 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
! uname -a | cut -d" " -f1,3,12  # получаем информацию конкретно про ядро

Linux 5.4.109+ x86_64


### Sed & Grep

grep позволяет фильтровать входной поток по указанному регулярному выражению
sed (sequence editor) - редактор входного потока. Делает примерно тоже, что и grep, однаок имеет свой не очень сложный формат задания правил редактирования. Возможные команды:

* `[диапазон]p` - напечатать диапазон строк
* `[диапазон]d` - удалить диапазон строкь
* `s/[паттерн1]/[паттерн2]` - заменить строку, подходящую под паттерн1 на паттерн2. Может комбинироваться с диапазоном. Также может добавляться ключ g (s/[паттерн1]/[паттерн2]/g) , позволяющая применять правила ко всему потоку сразу. Без этой опции ищется только первое совпадние с паттерном.

In [ ]:
! cat grades.csv

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"

In [ ]:
! cat grades.csv | grep Jim  # Ищем только джимов

"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"
"Dandy",     "Jim",        "087-75-4321", 47.0,     1.0,    23.0,    36.0,    45.0,   "C+"


In [ ]:
! cat grades.csv | grep "123\\-"

"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"


Команды для sed можно указывать в файле, однако если сами команды не слишком большие, то удобнее их указывать в виде аргументов командной строки. Для этого необходимо использовать флаг -e.

Также при использовании оператора p, необходимо указывать -n, чтобы не отображать лишние строки. В противном случае sed будет выводить вообще все исходные данные.

Подробнее можно узнать здесь - https://www.opennet.ru/docs/RUS/bash_scripting_guide/a14586.html

In [ ]:
! cat grades.csv | sed -n -e "3"p  # печатаем третью строку

"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"


In [ ]:
! cat grades.csv | sed -n -e "0~3"p | head # печатаем каждую третью строку, начиная с нулевой

"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Carnivore", "Art",        "565-89-0123", 44.0,     1.0,    80.0,    60.0,    40.0,   "D+"
"Franklin",  "Benny",      "234-56-2890", 50.0,     1.0,    90.0,    80.0,    90.0,   "B-"


In [ ]:
! cat grades.csv | sed -n -e "3,5"p  # печатаем с 3 по 5 строку


"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"


In [ ]:
! cat grades.csv | sed -n -e "6,$"p  # печатаем с 6 до конца

"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"
"Carnivore", "Art",        "565-89-0123", 44.0,     1.0,    80.0,    60.0,    40.0,   "D+"
"Dandy",     "Jim",        "087-75-4321", 47.0,     1.0,    23.0,    36.0,    45.0,   "C+"
"Elephant",  "Ima",        "456-71-9012", 45.0,     1.0,    78.0,    88.0,    77.0,   "B-"
"Franklin",  "Benny",      "234-56-2890", 50.0,     1.0,    90.0,    80.0,    90.0,   "B-"
"George",    "Boy",        "345-67-3901", 40.0,     1.0,    11.0,    -1.0,     4.0,   "B"
"H

In [ ]:
! cat grades.csv | sed -n -e "/Jim/"p  # печатаем все, которые подходят под паттерн (тоже что и grep)

"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"
"Dandy",     "Jim",        "087-75-4321", 47.0,     1.0,    23.0,    36.0,    45.0,   "C+"


In [ ]:
# Паттерны могут быть и более сложными. Например здесь паттерн ищем все Yankton ИЛИ NC
! cat grades.csv | sed -n -e "/Jim\|Fred/"p

"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"
"Dandy",     "Jim",        "087-75-4321", 47.0,     1.0,    23.0,    36.0,    45.0,   "C+"


In [ ]:
! cat grades.csv | sed -n -e "/Bif/,/Ima/"p  # поток от первой записи где есть Bif до записи с Ima

"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"
"Carnivore", "Art",        "565-89-0123", 44.0,     1.0,    80.0,    60.0,    40.0,   "D+"
"Dandy",     "Jim",        "087-75-4321", 47.0,     1.0,    23.0,    36.0,    45.0,   "C+"
"Elephant",  "Ima",        "456-71-9012", 45.0,     1.0,    78.0,    88.0,    77.0,   "B-"


Точно также можно пользоваться оператором удаления `d`

In [ ]:
! cat grades.csv | sed -e "/Bif/,/Ima/"d | head # удаляем все с Bif до Ima

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Franklin",  "Benny",      "234-56-2890", 50.0,     1.0,    90.0,    80.0,    90.0,   "B-"
"George",    "Boy",        "345-67-3901", 40.0,     1.0,    11.0,    -1.0,     4.0,   "B"


Оператором `s` можно заменять какие-то элементы потока. 

In [ ]:
! cat grades.csv | sed -e "s/Jim/Weierstrass/"

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Weierstrass",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0

Команда sed также имеет ключ -i позволяющий редактировать файлы "на месте". Например мы можем убрать заголовок у csv файла, не создавая новый, а отредактировав старый, что гораздо быстрее и менее затратно с точки зрения диска.

In [ ]:
! cp grades.csv grades-headless.csv

In [ ]:
! head grades-headless.csv

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"


In [ ]:
! sed -i grades-headless.csv -e '1'd

In [ ]:
! head grades-headless.csv

"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"
"Backus",    "Jim",        "143-12-1234", 48.0,     1.0,    97.0,    96.0,    97.0,   "A+"


### Jq
jq не является стандартной программой и ее необходимо самостоятельно установить. Для Ubuntu - apt-get install jq.

jq - это манипулятор JSON документами. Имеет свой язык запросов к JSON, чем то похожий на пайплайны в bash.

Подробнее узнать можно здесь - https://stedolan.github.io/jq/

В файле covid.json содержатся записи о заболеваниях короновирусом в различных странах.

In [ ]:
! apt-get update && apt-get install jq -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
! head covid.json

{
    "measures": [
        {
            "uid": 4,
            "country_iso2": "AF",
            "country_iso3": "AFG",
            "country_code": 4,
            "country": "Afghanistan",
            "combined_name": "Afghanistan",
            "population": 38928341,


Основной формат запроса в jq - это json path - путь из ключей, по которому нужно пройтись. Каждый такой path генерирует новый поток данных, согласно этому запросу, который можно дальше модифицировать.

Самый короткий запрос это . - то есть мы запрашиваем весть документ целиком. Попробуем посмотреть на первый элемент в массиве measures.

In [ ]:
! cat covid.json | jq '.measures[0]'

{
  "uid": 4,
  "country_iso2": "AF",
  "country_iso3": "AFG",
  "country_code": 4,
  "country": "Afghanistan",
  "combined_name": "Afghanistan",
  "population": 38928341,
  "loc": {
    "type": "Point",
    "coordinates": [
      67.71,
      33.9391
    ]
  },
  "confirmed": 0,
  "deaths": 0,
  "recovered": 0
}


In [ ]:
! cat covid.json | jq '.measures[0].country'

"Afghanistan"


In [ ]:
! cat covid.json | jq '. | keys'

[
  "measures"
]


In [ ]:
! cat covid.json | jq '.measures[0] | keys'

[
  "combined_name",
  "confirmed",
  "country",
  "country_code",
  "country_iso2",
  "country_iso3",
  "deaths",
  "loc",
  "population",
  "recovered",
  "uid"
]


In [ ]:
! cat covid.json | jq '.measures[0].loc | keys'

[
  "coordinates",
  "type"
]


In [ ]:
! cat covid.json | jq '.measures[3:5]'

[
  {
    "uid": 4,
    "country_iso2": "AF",
    "country_iso3": "AFG",
    "country_code": 4,
    "country": "Afghanistan",
    "combined_name": "Afghanistan",
    "population": 38928341,
    "loc": {
      "type": "Point",
      "coordinates": [
        67.71,
        33.9391
      ]
    },
    "confirmed": 0,
    "deaths": 0,
    "recovered": 0
  },
  {
    "uid": 4,
    "country_iso2": "AF",
    "country_iso3": "AFG",
    "country_code": 4,
    "country": "Afghanistan",
    "combined_name": "Afghanistan",
    "population": 38928341,
    "loc": {
      "type": "Point",
      "coordinates": [
        67.71,
        33.9391
      ]
    },
    "confirmed": 0,
    "deaths": 0,
    "recovered": 0
  }
]


In [ ]:
! cat covid.json | jq '.measures | length'  # В массиве 1000 элементов

1000


In [ ]:
! cat covid.json | jq '.measures[0:200] | .[].confirmed'

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
7
7
7
11
16
21
22
22
22
24
24
40
40
94
110
110
273
281
299
349
367
423
444
484
521
607
714
1828
2171
2335
2469
2704
2894
3224
3392
3563
3778
4033
4402
4687
4963
5639
6053
6402
6664
7072
7653
8145
8676
9216
9998
10582
11173
11831
13036
15750
16509
18054
18969
19551
20342
20917
21459
22142
22890
23546
24102
24766
25527
26310
26874
27532
27878
28424
33190
33384
33908
0
0
0
0
0
0
0
0
0
42
51
55
59
64
70
76
89
104
123
146
174
186
197
212
475
518
539
548
562
584
609
782
795
820
832
842
850
856
868
872
1122
1143
1788
1891
2114
2819
2893
2964
3038
3106
3188
3278
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
3
5
12
12
17
19
54
60
74
87
90
139
201
230
264
302
367
409
511
584
716
847
0
0
0
0
0
0
0
0


In [ ]:
! cat covid.json | jq '.measures[].country' | sort | uniq

"Afghanistan"
"Albania"
"Algeria"
"Andorra"
"Angola"
"Antigua and Barbuda"
"Argentina"


In [ ]:
! cat covid.json | jq -r '.measures[].country' | sort | uniq

Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina


С помощью jq можно фильтровать запросы. Для этого есть оператор select

In [ ]:
! cat covid.json | jq '.measures[] | select(.country == "Angola")' | head -n 30

{
  "uid": 24,
  "country_iso2": "AO",
  "country_iso3": "AGO",
  "country_code": 24,
  "country": "Angola",
  "combined_name": "Angola",
  "population": 32866268,
  "loc": {
    "type": "Point",
    "coordinates": [
      17.8739,
      -11.2027
    ]
  },
  "confirmed": 0,
  "deaths": 0,
  "recovered": 0
}
{
  "uid": 24,
  "country_iso2": "AO",
  "country_iso3": "AGO",
  "country_code": 24,
  "country": "Angola",
  "combined_name": "Angola",
  "population": 32866268,
  "loc": {
    "type": "Point",
    "coordinates": [


In [ ]:
! cat covid.json | jq '.measures[] | select(.country == "Angola") | .confirmed'

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
7
7
8
8
8
10
14
17
19
19
19
19
19
19
19
24
24
24
24
25
25
25
25
26
27
27
27
30
35
35
35
36
36
36
43
43
45
45
45
45
48
48
48
48
84
86
86
86
86
86
176
346
346
386
386
396
458
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
4
4
5
7
16
19
19
19
27
50
52
52
58
60
61
69
70
70
71
74
81
86
88
91
92
96
113
118
130
138
140
142
148
155
166
172
183
186
189
197
212
212
259
267
276
284
291
315
328
346


### Archives
Очень часто данные хранятся в виде архивов. Команды tar и zip\unzip позволяют распаковывать архивы.

tar имеет целый набор однобуквенных ключей, комбинация которых позволяет производить различные операции над архивами.

`c` - создать архив

`x` - распаковать архив

`z` - использовать алгоритм gzip. Архивы, созданные с таким алгоритмом, имеют расширение .tar.gz

`v` - печатать на экран детали распаковки

`f` - считать архив из указанного файла

zip\unzip работает немного проще. Команда zip создает новый архив, команда unzip распаковывает указанный архив.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=tar&category=1 и здесь - https://www.opennet.ru/man.shtml?topic=unzip&category=1&russian=4

In [ ]:
# Создадим архив из файла cities
! tar -czvf grades.tar.gz grades.csv

grades.csv


In [ ]:
! head -n 2 grades.tar.gz

2��?�OO;Ֆz���81%�N=����0݌H�V���_��Vz�VŮ����ڡ+�A�)�
e� (  

In [ ]:
# Распакуем этот архив в новую директорию tar-grades
! mkdir -p tar-grades && tar -xzvf grades.tar.gz -C tar-grades/

grades.csv


In [ ]:
! ls tar-grades/

grades.csv


In [ ]:
! head tar-grades/grades.csv

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"


In [ ]:
! apt-get install zip -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.


In [ ]:
# Сделаем точно тоже самое, но с помощью zip\unzip
! zip grades.zip grades.csv

  adding: grades.csv (deflated 64%)


In [ ]:
! head -n 2 grades.zip

PK    h�u>a')�+  	  
׏���p�ut]�L3����4�۽J_$1���tz�NZ��3����M�QN7���NR��& ͠ȗ+� ��Z� ����(]ǡ�r���Z{.]���p J��Θ�Oj��5�֊���uq�p�aJR�2M��)���i�A�ݣ��Q_��Y�Nl�1�r�


In [ ]:
! mkdir -p zip-grades && yes | unzip grades.zip -d zip-grades/

Archive:  grades.zip
  inflating: zip-grades/grades.csv   


In [ ]:
! ls zip-grades/

grades.csv


In [ ]:
! head zip-grades/grades.csv

"Last name", "First name", "SSN",        "Test1", "Test2", "Test3", "Test4", "Final", "Grade"
"Alfalfa",   "Aloysius",   "123-45-6789", 40.0,    90.0,   100.0,    83.0,    49.0,   "D-"
"Alfred",    "University", "123-12-1234", 41.0,    97.0,    96.0,    97.0,    48.0,   "D+"
"Gerty",     "Gramma",     "567-89-0123", 41.0,    80.0,    60.0,    40.0,    44.0,   "C"
"Android",   "Electric",   "087-65-4321", 42.0,    23.0,    36.0,    45.0,    47.0,   "B-"
"Bumpkin",   "Fred",       "456-78-9012", 43.0,    78.0,    88.0,    77.0,    45.0,   "A-"
"Rubble",    "Betty",      "234-56-7890", 44.0,    90.0,    80.0,    90.0,    46.0,   "C-"
"Noshow",    "Cecil",      "345-67-8901", 45.0,    11.0,    -1.0,     4.0,    43.0,   "F"
"Buff",      "Bif",        "632-79-9939", 46.0,    20.0,    30.0,    40.0,    50.0,   "B+"
"Airpump",   "Andrew",     "223-45-6789", 49.0      1.0,    90.0,   100.0,    83.0,   "A"


Также достаточно популярен формат архивов `7z`. Чтобы начать работать с ним, необходимо поставить дополнительно специальный пакет - `apt-get install p7zip-full`.


In [ ]:
! apt-get update && apt-get install p7zip-full -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
! 7z a grades.7z grades.csv  # Создаем архив


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           1 file, 1545 bytes (2 KiB)

Creating archive: grades.7z

Items to compress: 1

  0%    
Files read from disk: 1
Archive size: 651 bytes (1 KiB)
Everything is Ok


In [ ]:
! head -n 2 grades.7z

7z��' L>	      b       �?c@�] '3�����'
��t)��-(�u� "1bc����'<���e_�F�|C�'@|���ݪ�B��ϛ�v������3�O��ާ��}�H ��+�Wʎ��� gA��������c�^���y=��75+p�Yj�&WS76���?�G��W��I�e�֞j�UYU��XGER�	�|����Ez�u�ߏdyo\�N���m�f�z��.P��*�⎈�������Y�!���� �����7��5�3�ЯG�����'��5��s�f�D`��m3��O"P��o3�'Ǖ��AP���R�!�8���O+Vӽ�i���Z�����_��޷�|���V��aI�r��ĝ���c��3�ܐ��HH&}�0�� Yر����8���a��O.#m:��(�Q ����L{{�u^�\�r��X�%��¹!t}��� �ȿ�$�vY!(�N��}�


In [ ]:
! mkdir -p 7z-grades && 7z x grades.7z -o7z-grades/  #  Распаковываем


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 651 bytes (1 KiB)

Extracting archive: grades.7z
--
Path = grades.7z
Type = 7z
Physical Size = 651
Headers Size = 130
Method = LZMA2:12
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       1545
Compressed: 651


In [ ]:
! ls 7z-grades/

grades.csv


### Networking
wget и curl позволяют выгружать данные из интернета.

wget более продвинутый - он умеет скачивать сразу множество файлов, поддерживает докачку файлов и так далее. curl более простой и может использоваться скорее для точечных запросов.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wget&category=1&russian=0 и здесь - https://www.opennet.ru/man.shtml?topic=curl&category=1&russian=3



In [ ]:
! apt-get install curl -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.


In [ ]:
! curl -L http://yandex.ru > yandex-curl.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  139k    0  139k    0     0  73557      0 --:--:--  0:00:01 --:--:--  267k


In [ ]:
! wget http://yandex.ru -O yandex-wget.html

--2021-05-19 20:30:05--  http://yandex.ru/
Resolving yandex.ru (yandex.ru)... 77.88.55.77, 5.255.255.80, 5.255.255.77, ...
Connecting to yandex.ru (yandex.ru)|77.88.55.77|:80... connected.
HTTP request sent, awaiting response... 302 Moved temporarily
Location: https://yandex.ru/ [following]
--2021-05-19 20:30:06--  https://yandex.ru/
Connecting to yandex.ru (yandex.ru)|77.88.55.77|:443... connected.
HTTP request sent, awaiting response... 200 Ok
Length: unspecified [text/html]
Saving to: ‘yandex-wget.html’

yandex-wget.html        [   <=>              ] 134.52K   238KB/s    in 0.6s    

2021-05-19 20:30:07 (238 KB/s) - ‘yandex-wget.html’ saved [137748]



In [ ]:
! head -c 100 yandex-curl.html

<!DOCTYPE html><html class="i-ua_js_no i-ua_css_standart i-ua_browser_unknown i-ua_browser-engine_un

In [ ]:
! head -c 100 yandex-wget.html

<!DOCTYPE html><html class="i-ua_js_no i-ua_css_standart i-ua_browser_ i-ua_browser-engine_ i-ua_bro

Создатим список ссылок и скачаем их все разом с помощью ключа -i



In [ ]:
%%writefile link-list.txt
http://yandex.ru
http://google.com

Writing link-list.txt


In [ ]:
! wget -i link-list.txt

--2021-05-19 20:30:08--  http://yandex.ru/
Resolving yandex.ru (yandex.ru)... 77.88.55.77, 5.255.255.80, 5.255.255.77, ...
Connecting to yandex.ru (yandex.ru)|77.88.55.77|:80... connected.
HTTP request sent, awaiting response... 302 Moved temporarily
Location: https://yandex.ru/ [following]
--2021-05-19 20:30:09--  https://yandex.ru/
Connecting to yandex.ru (yandex.ru)|77.88.55.77|:443... connected.
HTTP request sent, awaiting response... 200 Ok
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [   <=>              ] 133.89K   235KB/s    in 0.6s    

2021-05-19 20:30:10 (235 KB/s) - ‘index.html’ saved [137107]

--2021-05-19 20:30:10--  http://google.com/
Resolving google.com (google.com)... 74.125.142.113, 74.125.142.102, 74.125.142.139, ...
Connecting to google.com (google.com)|74.125.142.113|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.google.com/ [following]
--2021-05-19 20:30:10--  http://www.goo